In [11]:
import cv2
import numpy as np
import math

class Vec:
    def __init__(self, n):
        self.N = n
        self.p = [0 for y in range(n)]
        
    def make_uint(self):
        sum_p = 0.0
        for i in range(self.N):
            sum_p = sum_p + self.p[i] * self.p[i]
        sum_p = math.sqrt(sum_p)
        if sum_p > 0.0:
            for i in range(self.N):
                self.p[i] = self.p[i] / sum_p

    def normalization(self):
        sum_p = 0.0
        for i in range(self.N):
            sum_p = sum_p + self.p[i] * self.p[i]
        sum_p = math.sqrt(sum_p)
        return sum_p
        
    def zero(self):
        for i in range(self.N):
            self.p[i] = 0.0
        

class FDoG:
    def __init__(self):
        self.PI = 3.1415926

    def gauss(self, x, mean, sigma):
        return (math.exp((-(x - mean) * (x - mean)) / (2.0 * sigma * sigma) ) / math.sqrt(self.PI * 2.0 * sigma * sigma));

    def MakeGaussianVector(self, sigma):
        threshold = 0.001;
        
        i = 0
        while(True):
            i = i + 1
            if self.gauss(i, 0.0, sigma) < threshold:
                break
        GAU = Vec(i + 1)
        GAU.zero()
        
        GAU.p[0] = self.gauss(0.0, 0.0, sigma)
        for j in range(1, GAU.N):
            GAU.p[j] = self.gauss(j, 0.0, sigma)
            
        return GAU

    def GetDirectionalDoG(self, image, e, GAU1, GAU2, tau):
        vn = Vec(2)
        
        half_w1 = GAU1.N - 1
        half_w2 = GAU2.N - 1
        
        image_x = image.shape[0]
        image_y = image.shape[1]
        
        dog = [[0 for x in range(image_y)] for y in range(image_x)]
        
        for i in range(image_x):
            for j in range(image_y):
                sum1 = sum2 = 0.0
                w_sum1 = w_sum2 = 0.0
                weight1 = weight2 = 0.0
                
                vn.p[0] = -e.p[i][j].ty
                vn.p[1] = e.p[i][j].tx
                
                if vn.p[0] == 0.0 and vn.p[1] == 0.0:
                    sum1 = 255.0
                    sum2 = 255.0
                    dog[i][j] = sum1 - tau * sum2;
                    continue
                    
                d_x = i
                d_y = j
                
                for s in range(-half_w2, half_w2 + 1):
                    x = d_x + vn.p[0] * s
                    y = d_y + vn.p[1] * s
                    
                    if x > image_x - 1.0 or x < 0.0 or y > image_y - 1.0 or y < 0.0:
                        continue
                        
                    x1 = int(round(x))
                    if x1 < 0:
                        x1 = 0
                    if x1 > image_x - 1:
                        x1 = image_x - 1

                    y1 = int(round(y))
                    if y1 < 0:
                        y1 = 0
                    if y1 > image_y - 1:
                        y1 = image_y - 1
                        
                    val = image[x1][y1]
                    
                    dd = abs(s);
                    
                    weight1 = 0.0 if dd > half_w1 else GAU1.p[dd]
                    '''
                    if dd > half_w1:
				        weight1 = 0.0
				    else:
				        weight1 = GAU1.p[dd]
				    '''
                    sum1 = sum1 + val * weight1
                    w_sum1 = w_sum1 + weight1
				    
                    weight2 = GAU2.p[dd]
                    sum2 = sum2 + val * weight2
                    w_sum2 = w_sum2 + weight2
				    
                sum1 = sum1 / w_sum1
                sum2 = sum2 / w_sum2
				
                dog[i][j] = sum1 - tau * sum2
        return dog
        
    def GetFlowDoG(self, e, dog, GAU3):
        vt = Vec(2)
        
        image_x = len(dog)
        image_y = len(dog[0])
        
        tmp = [[0 for x in range(image_y)] for y in range(image_x)]
        
        half_l = GAU3.N - 1
        step_size = 1.0
        
        for i in range(image_x):
            for j in range(image_y):
                sum1 = 0.0
                w_sum1 = 0.0
                weight1 = 0.0
                
                val = dog[i][j]
                weight1 = GAU3.p[0]
                sum1 = val * weight1
                w_sum1 = w_sum1 + weight1
                
                d_x = i_x = i
                d_y = i_y = j
                
                for k in range(half_l):
                    vt.p[0] = e.p[i_x][i_y].tx
                    vt.p[1] = e.p[i_x][i_y].ty
                    if vt.p[0] == 0.0 and vt.p[1] == 0.0:
                        break
                    x = d_x
                    y = d_y
                    
                    if x > image_x - 1.0 or x < 0.0 or y > image_y - 1.0 or y < 0.0:
                        break
                    x1 = int(round(x))
                    if x1 < 0:
                        x1 = 0
                    if x1 > image_x - 1:
                        x1 = image_x - 1
                    
                    y1 = int(round(y))
                    if y1 < 0:
                        y1 = 0
                    if y1 > image_y - 1:
                        y1 = image_y - 1
                    
                    val = dog[x1][y1]
                    
                    weight1 = GAU3.p[k]
                    
                    sum1 = sum1 + val * weight1
                    w_sum1 = w_sum1 + weight1
                    
                    d_x = d_x + vt.p[0] * step_size; 
                    d_y = d_y + vt.p[1] * step_size; 
                    
                    i_x = int(round(d_x))
                    i_y = int(round(d_y))
                    if d_x < 0 or d_x > image_x - 1 or d_y < 0 or d_y > image_y - 1:
                        break
                d_x = i_x = i
                d_y = i_y = j
                
                for k in range(half_l):
                    vt.p[0] = -e.p[i_x][i_y].tx
                    vt.p[1] = -e.p[i_x][i_y].ty
                    if vt.p[0] == 0.0 and vt.p[1] == 0.0:
                        break
                    x = d_x
                    y = d_y
                    
                    if x > image_x - 1 or x < 0.0 or y > image_y - 1 or y < 0.0:
                        break;
                    x1 = int(round(x))
                    if x1 < 0:
                        x1 = 0
                    if x1 > image_x - 1:
                        x1 = image_x - 1
                        
                    y1 = int(round(y))
                    if y1 < 0:
                        y1 = 0
                    if y1 > image_y - 1:
                        y1 = image_y - 1
                    
                    val = dog[x1][y1]
                    weight1 = GAU3.p[k] 
                    
                    sum1 = sum1 + val * weight1
                    w_sum1 = w_sum1 + weight1
                    
                    d_x = d_x + vt.p[0] * step_size
                    d_y = d_y + vt.p[1] * step_size
                    
                    i_x = int(round(d_x))
                    i_y = int(round(d_y))
                    if d_x < 0 or d_x > image_x - 1 or d_y < 0 or d_y > image_y - 1:
                        break
                sum1 = sum1 / w_sum1; 

                tmp[i][j] = 1.0 if sum1 > 0 else 1.0 + np.tanh(sum1)
                '''
                if sum1 > 0:
                    tmp[i][j] = 1.0
                else:
                    tmp[i][j] = 1.0 + tanh(sum1)
                '''
        return tmp

    def getFDoG(self, image, e, sigma, sigma_3, tau):
        image_x = image.shape[0]
        image_y = image.shape[1]
        
        GAU1 = self.MakeGaussianVector(sigma)
        GAU2 = self.MakeGaussianVector(sigma * 1.6)
        half_w1 = GAU1.N - 1
        half_w2 = GAU2.N - 1
        
        GAU3 = self.MakeGaussianVector(sigma_3)
        half_l = GAU3.N - 1
        
        dog = self.GetDirectionalDoG(image, e, GAU1, GAU2, tau)
        tmp = self.GetFlowDoG(e, dog, GAU3)
        
        for i in range(image_x):
            for j in range(image_y):
                image[i][j] = int(round(tmp[i][j] * 255.0))
                
        return image
    
    def GrayThresholding(self, image, thres):
        image_x = image.shape[0]
        image_y = image.shape[1]
        
        for i in range(image_x):
            for j in range(image_y):
                val = image[i][j] / 255.0
                image[i][j] = int(round(val * 255.0)) if val < thres else 255
                
        return image

In [12]:
import cv2
import numpy as np
import math

class Vect:
    def __init__(self):
        self.tx = 1.0
        self.ty = 0.0
        self.mag = 1.0

class ETF:
    def __init__(self, Nr=3, Nc=2):
        self.Nr = Nr
        self.Nc = Nc
        self.max_grad = 1.0
        self.p = [[0 for x in range(Nc)] for y in range(Nr)]
        
        for x in range(Nr):
            for y in range(Nc):
                self.p[x][y] = Vect()

    def copy_ETF(self, etf):
        for i in range(self.Nr):
            for j in range(self.Nc):
                self.p[i][j].tx = etf.p[i][j].tx
                self.p[i][j].ty = etf.p[i][j].ty
                self.p[i][j].mag = etf.p[i][j].mag
        self.max_grad = etf.max_grad

    def make_uint(self, vx, vy):
        mag = math.sqrt(vx * vx + vy * vy)
        if mag != 0.0:
            vx = vx / mag
            vy = vy / mag
        return (vx, vy)
                
    def normalize(self):
        for i in range(self.Nr):
            for j in range(self.Nc):
                self.p[i][j].tx, self.p[i][j].ty = self.make_uint(self.p[i][j].tx, self.p[i][j].ty)
                '''
                mag = math.sqrt(self.p[i][j].tx * self.p[i][j].tx + self.p[i][j].ty * self.p[i][j].ty)
                if mag != 0.0:
                    self.p[i][j].tx = self.p[i][j].tx / mag
                    self.p[i][j].ty = self.p[i][j].ty / mag
                '''
                self.p[i][j].mag = self.p[i][j].mag / self.max_grad
                            

    def setFlow(self, image, MAX_VAL = 1020.0):
        self.max_grad = -1.0
        
        v = [0.0, 0.0]
        
        for i in range(1, self.Nr - 1):
            for j in range(1, self.Nc - 1):
                self.p[i][j].tx = (image[i+1][j-1] + 2.0*image[i+1][j] + image[i+1][j+1] - image[i-1][j-1] - 2.0*image[i-1][j] - image[i-1][j+1]) / MAX_VAL
                self.p[i][j].ty = (image[i-1][j+1] + 2.0*image[i][j+1] + image[i+1][j+1] - image[i-1][j-1] - 2.0*image[i][j-1] - image[i+1][j-1]) / MAX_VAL
                v[0] = self.p[i][j].tx
                v[1] = self.p[i][j].ty
                self.p[i][j].tx = -v[1]
                self.p[i][j].ty = v[0]
                
                self.p[i][j].mag = math.sqrt(self.p[i][j].tx * self.p[i][j].tx + self.p[i][j].ty * self.p[i][j].ty)
                
                if self.p[i][j].mag > self.max_grad:
                    self.max_grad = self.p[i][j].mag;
        
        for i in range(1, self.Nr - 2):
            self.p[i][0].tx = self.p[i][1].tx
            self.p[i][0].ty = self.p[i][1].ty
            self.p[i][0].mag = self.p[i][1].mag;
            self.p[i][self.Nc - 1].tx = self.p[i][self.Nc - 2].tx
            self.p[i][self.Nc - 1].ty = self.p[i][self.Nc - 2].ty
            self.p[i][self.Nc - 1].mag = self.p[i][self.Nc - 2].mag

        for j in range(1, self.Nc - 2):
            self.p[0][j].tx = self.p[1][j].tx
            self.p[0][j].ty = self.p[1][j].ty
            self.p[0][j].mag = self.p[1][j].mag
            self.p[self.Nr - 1][j].tx = self.p[self.Nr - 2][j].tx
            self.p[self.Nr - 1][j].ty = self.p[self.Nr - 2][j].ty
            self.p[self.Nr - 1][j].mag = self.p[self.Nr - 2][j].mag
            
        self.p[0][0].tx = ( self.p[0][1].tx + self.p[1][0].tx ) / 2;
        self.p[0][0].ty = ( self.p[0][1].ty + self.p[1][0].ty ) / 2;
        self.p[0][0].mag = ( self.p[0][1].mag + self.p[1][0].mag ) / 2;
        self.p[0][self.Nc-1].tx = ( self.p[0][self.Nc-2].tx + self.p[1][self.Nc-1].tx ) / 2;
        self.p[0][self.Nc-1].ty = ( self.p[0][self.Nc-2].ty + self.p[1][self.Nc-1].ty ) / 2;
        self.p[0][self.Nc-1].mag = ( self.p[0][self.Nc-2].mag + self.p[1][self.Nc-1].mag ) / 2;
        self.p[self.Nr-1][0].tx = ( self.p[self.Nr-1][1].tx + self.p[self.Nr-2][0].tx ) / 2;
        self.p[self.Nr-1][0].ty = ( self.p[self.Nr-1][1].ty + self.p[self.Nr-2][0].ty ) / 2;
        self.p[self.Nr-1][0].mag = ( self.p[self.Nr-1][1].mag + self.p[self.Nr-2][0].mag ) / 2;
        self.p[self.Nr - 1][self.Nc - 1].tx = ( self.p[self.Nr - 1][self.Nc - 2].tx + self.p[self.Nr - 2][self.Nc - 1].tx ) / 2;
        self.p[self.Nr - 1][self.Nc - 1].ty = ( self.p[self.Nr - 1][self.Nc - 2].ty + self.p[self.Nr - 2][self.Nc - 1].ty ) / 2;
        self.p[self.Nr - 1][self.Nc - 1].mag = ( self.p[self.Nr - 1][self.Nc - 2].mag + self.p[self.Nr - 2][self.Nc - 1].mag ) / 2;
        
        self.normalize()
    
    def smooth(self, half_w, M):
        MAX_GRADIENT = -1
        
        image_x = self.Nr
        image_y = self.Nc
        
        e2 = ETF(image_x, image_y)
        e2.copy_ETF(self)
        
        g = [0, 0]
        v = [0, 0]
        w = [0, 0]
        
        for k in range(M):
            for j in range(image_y):
                for i in range(image_x):
                    g[0] = g[1] = 0.0
                    v[0] = self.p[i][j].tx
                    v[1] = self.p[i][j].ty
                    for s in range(-half_w, half_w + 1):
                        x = i + s
                        y = j
                        if x > image_x - 1:
                            x = image_x - 1
                        elif x < 0:
                            x = 0
                        
                        if y > image_y - 1:
                            y = image_y - 1
                        elif y < 0:
                            y = 0
                            
                        mag_diff = self.p[x][y].mag - self.p[i][j].mag
                        
                        w[0] = self.p[x][y].tx
                        w[1] = self.p[x][y].ty
                        
                        factor = 1.0
                        angle = v[0] * w[0] + v[1] * w[1]
                        if angle < 0.0:
                            factor = -1.0
                            
                        weight = mag_diff + 1
                        g[0] = g[0] + weight * self.p[x][y].tx * factor
                        g[1] = g[1] + weight * self.p[x][y].ty * factor

                    g[0], g[1] = self.make_uint(g[0], g[1])
                    '''
                    mag = math.sqrt(g[0] * g[0] + g[1] * g[1])
                    if mag != 0.0:
                        g[0] = g[0] / mag
                        g[1] = g[1] / mag
                    '''
                    e2.p[i][j].tx = g[0]
                    e2.p[i][j].ty = g[1]
            self.copy_ETF(e2)
            
            for j in range(image_y):
                for i in range(image_x):
                    g[0] = g[1] = 0.0
                    v[0] = self.p[i][j].tx
                    v[1] = self.p[i][j].ty
                    for t in range(-half_w, half_w + 1):
                        x = i
                        y = j + t
                        if x > image_x - 1:
                            x = image_x - 1
                        elif x < 0:
                            x = 0
                        
                        if y > image_y - 1:
                            y = image_y - 1
                        elif y < 0:
                            y = 0
                            
                        mag_diff = self.p[x][y].mag - self.p[i][j].mag
                        
                        w[0] = self.p[x][y].tx
                        w[1] = self.p[x][y].ty
                        
                        factor = 1.0
                        angle = v[0] * w[0] + v[1] * w[1]
                        if angle < 0.0:
                            factor = -1.0
                            
                        weight = mag_diff + 1
                        g[0] = g[0] + weight * self.p[x][y].tx * factor
                        g[1] = g[1] + weight * self.p[x][y].ty * factor
                        
                    g[0], g[1] = self.make_uint(g[0], g[1])
                    '''
                    mag = math.sqrt(g[0] * g[0] + g[1] * g[1])
                    if mag != 0.0:
                        g[0] = g[0] / mag
                        g[1] = g[1] / mag
                    '''
                    e2.p[i][j].tx = g[0]
                    e2.p[i][j].ty = g[1]
            self.copy_ETF(e2)

In [5]:
!pwd

/Users/Nagarjuna/SketchBack


In [20]:
def getFDoGImg(image):
    print ("FDoG start")
    img_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    e = ETF(img_gray.shape[0], img_gray.shape[1])
    e.setFlow(img_gray)
    e.smooth(4, 2)
    
    fdog = FDoG()
    FDoGImg = fdog.getFDoG(img_gray, e, 1.0, 3.0, 0.97)
    FDoGImg = fdog.GrayThresholding(FDoGImg, 0.7)
    cv2.imwrite("CharminarFDog.png", FDoGImg)
    print ("FDoG finish")
    return FDoGImg

In [21]:
img = cv2.imread('Charminar.jpg')
FDoGImage = getFDoGImg(img)

FDoG start
FDoG finish
